In [5]:
import numpy as np
import torch 
import gdown

file_id = "19MBh9JIJTxYIPAeO7G5RML5_ddjJ1Cpa"
output_path = "downloaded.zip" 
gdown.download(f"https://drive.google.com/uc?export=download&id={file_id}", output_path, quiet=False)

ModuleNotFoundError: No module named 'gdown'